# Advanced K-Means Clustering from Scratch and Comparison with scikit-learn

This notebook implements a highly challenging K-Means clustering algorithm from scratch in Python, explains the code step-by-step, and compares it against scikit-learn's implementation.

## 1. Import Required Libraries

```python
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans as SklearnKMeans
from sklearn.metrics import silhouette_score, adjusted_rand_score
import random
```

## 2. Generate Complex Synthetic Dataset

## 2. Generate Complex Synthetic Dataset
Create a synthetic dataset with overlapping clusters, varying densities, and outliers to challenge the K-Means algorithm.

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)

def generate_complex_dataset(n_samples=1000, n_features=2, n_clusters=5):
    """
    Generate a complex dataset with overlapping clusters, varying densities, and outliers.
    
    Parameters:
    -----------
    n_samples : int
        Number of samples in the dataset
    n_features : int
        Number of features (dimensions)
    n_clusters : int
        Number of true clusters to generate
        
    Returns:
    --------
    X : numpy.ndarray
        Generated dataset
    true_labels : numpy.ndarray
        True cluster labels for each data point
    """
    # Initialize arrays to store data and labels
    X = np.zeros((n_samples, n_features))
    true_labels = np.zeros(n_samples, dtype=int)
    
    # Track current position in the dataset
    current_pos = 0
    
    # Create clusters with varying densities and spreads
    for i in range(n_clusters):
        # Determine number of samples in this cluster (varying sizes)
        if i == n_clusters - 1:
            cluster_samples = n_samples - current_pos
        else:
            # Random cluster size, weighted toward the beginning clusters
            weight = 1.0 - 0.5 * (i / n_clusters)
            cluster_samples = int((n_samples / n_clusters) * weight)
            cluster_samples = min(cluster_samples, n_samples - current_pos - (n_clusters - i - 1))
        
        # Generate cluster center
        center = np.random.uniform(-10, 10, size=n_features)
        
        # Vary the spread of each cluster
        spread = np.random.uniform(0.5, 2.5)
        
        # Generate cluster data with normal distribution
        cluster_data = np.random.normal(loc=center, scale=spread, size=(cluster_samples, n_features))
        
        # Add the cluster data to our dataset
        X[current_pos:current_pos + cluster_samples] = cluster_data
        true_labels[current_pos:current_pos + cluster_samples] = i
        
        current_pos += cluster_samples
    
    # Add some outliers (random points far from all clusters)
    n_outliers = int(n_samples * 0.05)  # 5% outliers
    outlier_indices = np.random.choice(n_samples, n_outliers, replace=False)
    
    for idx in outlier_indices:
        # Generate random point far from the center
        X[idx] = np.random.uniform(-20, 20, size=n_features)
        # Label outliers as -1 (will be helpful for evaluation)
        true_labels[idx] = -1
    
    # Introduce some overlap between clusters
    n_overlap = int(n_samples * 0.1)  # 10% overlap
    overlap_indices = np.random.choice(n_samples, n_overlap, replace=False)
    
    for idx in overlap_indices:
        if true_labels[idx] != -1:  # Don't overlap outliers
            # Move this point toward a neighboring cluster
            target_cluster = (true_labels[idx] + 1) % n_clusters
            # Find center of target cluster
            target_points = X[true_labels == target_cluster]
            if len(target_points) > 0:
                target_center = np.mean(target_points, axis=0)
                # Move 70% of the way from original cluster to target cluster
                original_pos = X[idx].copy()
                X[idx] = original_pos + 0.7 * (target_center - original_pos)
    
    return X, true_labels

# Generate dataset
X, true_labels = generate_complex_dataset(n_samples=1000, n_features=2, n_clusters=5)

# Visualize the generated dataset
plt.figure(figsize=(10, 8))
plt.scatter(X[:, 0], X[:, 1], c=true_labels, cmap='viridis', s=30, alpha=0.7)
plt.title('Generated Complex Dataset with 5 Clusters')
plt.xlabel('Feature 1')
plt.ylabel('Feature 2')
plt.colorbar(label='Cluster')
plt.grid(True, alpha=0.3)
plt.show()

print(f"Dataset shape: {X.shape}")
print(f"Number of unique clusters: {len(np.unique(true_labels))}")
print(f"Cluster sizes: {np.bincount(true_labels[true_labels >= 0])}")
print(f"Number of outliers: {np.sum(true_labels == -1)}")

## 3. Implement Custom K-Means Algorithm from Scratch

Below we'll implement a custom K-Means algorithm from scratch, including:
- Random centroid initialization
- Distance calculation
- Cluster assignment
- Centroid updating
- Convergence checking
- Handling edge cases

In [ ]:
class KMeans:
    """
    Custom implementation of K-Means clustering algorithm.
    
    Parameters:
    -----------
    n_clusters : int, default=8
        The number of clusters to form.
    
    max_iter : int, default=300
        Maximum number of iterations for a single run.
    
    tol : float, default=1e-4
        Relative tolerance with regards to inertia to declare convergence.
    
    n_init : int, default=10
        Number of times the algorithm will be run with different centroid seeds.
        
    init : str, default='random'
        Method for initialization ('random' or 'kmeans++').
        
    random_state : int, default=None
        Determines random number generation for centroid initialization.
    
    Attributes:
    -----------
    centroids : numpy.ndarray
        Coordinates of cluster centers.
    
    labels : numpy.ndarray
        Labels of each point.
    
    inertia : float
        Sum of squared distances of samples to their closest cluster center.
        
    n_iter : int
        Number of iterations run.
    """
    
    def __init__(self, n_clusters=8, max_iter=300, tol=1e-4, n_init=10, 
                 init='random', random_state=None):
        self.n_clusters = n_clusters
        self.max_iter = max_iter
        self.tol = tol
        self.n_init = n_init
        self.init = init
        self.random_state = random_state
        
        # Attributes that will be set during fitting
        self.centroids = None
        self.labels = None
        self.inertia = None
        self.n_iter = None
    
    def _init_centroids(self, X):
        """
        Initialize centroids either randomly or using k-means++.
        
        Parameters:
        -----------
        X : numpy.ndarray
            Training data.
            
        Returns:
        --------
        centroids : numpy.ndarray
            Initial positions of centroids.
        """
        if self.random_state is not None:
            np.random.seed(self.random_state)
        
        n_samples, n_features = X.shape
        
        if self.init == 'random':
            # Random initialization: Choose random data points as initial centroids
            idx = np.random.choice(n_samples, self.n_clusters, replace=False)
            centroids = X[idx].copy()
        elif self.init == 'kmeans++':
            # KMeans++ initialization will be implemented separately
            centroids = self._kmeans_plus_plus_init(X)
        else:
            raise ValueError("Invalid init method. Use 'random' or 'kmeans++'")
        
        return centroids
    
    def _assign_clusters(self, X, centroids):
        """
        Assign each data point to the nearest centroid.
        
        Parameters:
        -----------
        X : numpy.ndarray
            Training data.
        centroids : numpy.ndarray
            Current centroid positions.
            
        Returns:
        --------
        labels : numpy.ndarray
            Cluster assignments for each data point.
        distances : numpy.ndarray
            Distance from each point to its assigned centroid.
        """
        n_samples = X.shape[0]
        
        # Initialize arrays to store distances and labels
        distances = np.zeros((n_samples, self.n_clusters))
        
        # Calculate Euclidean distance from each point to each centroid
        for k in range(self.n_clusters):
            # Vectorized distance calculation
            diff = X - centroids[k]
            sq_dist = np.sum(diff**2, axis=1)
            distances[:, k] = sq_dist
        
        # Assign each point to the nearest centroid
        labels = np.argmin(distances, axis=1)
        
        # Get the distances to the assigned centroids
        min_distances = np.min(distances, axis=1)
        
        return labels, min_distances
    
    def _update_centroids(self, X, labels):
        """
        Update centroid positions based on assigned clusters.
        
        Parameters:
        -----------
        X : numpy.ndarray
            Training data.
        labels : numpy.ndarray
            Current cluster assignments.
            
        Returns:
        --------
        new_centroids : numpy.ndarray
            Updated centroid positions.
        empty_clusters : list
            Indices of any empty clusters.
        """
        n_features = X.shape[1]
        new_centroids = np.zeros((self.n_clusters, n_features))
        empty_clusters = []
        
        for k in range(self.n_clusters):
            # Get all points assigned to cluster k
            cluster_points = X[labels == k]
            
            if len(cluster_points) == 0:
                # Handle empty cluster
                empty_clusters.append(k)
                # Keep the old centroid for now
                new_centroids[k] = self.centroids[k]
            else:
                # Update centroid to mean of all points in the cluster
                new_centroids[k] = np.mean(cluster_points, axis=0)
        
        return new_centroids, empty_clusters
    
    def _handle_empty_clusters(self, X, labels, empty_clusters):
        """
        Handle empty clusters by assigning a new centroid.
        
        Parameters:
        -----------
        X : numpy.ndarray
            Training data.
        labels : numpy.ndarray
            Current cluster assignments.
        empty_clusters : list
            Indices of empty clusters.
            
        Returns:
        --------
        centroids : numpy.ndarray
            Updated centroids with new positions for empty clusters.
        """
        centroids = self.centroids.copy()
        
        for k in empty_clusters:
            # Strategy: Find the point furthest from its centroid
            # and make it the new centroid for the empty cluster
            
            # Get distances from each point to its assigned centroid
            distances = np.zeros(X.shape[0])
            for i, x in enumerate(X):
                assigned_cluster = labels[i]
                distances[i] = np.sum((x - centroids[assigned_cluster])**2)
            
            # Find the point furthest from its assigned centroid
            furthest_point_idx = np.argmax(distances)
            
            # Assign this point as the new centroid
            centroids[k] = X[furthest_point_idx]
            
            # Update the point's cluster assignment
            labels[furthest_point_idx] = k
        
        return centroids, labels
    
    def _calculate_inertia(self, X, labels, centroids):
        """
        Calculate the inertia (sum of squared distances to centroids).
        
        Parameters:
        -----------
        X : numpy.ndarray
            Training data.
        labels : numpy.ndarray
            Cluster assignments.
        centroids : numpy.ndarray
            Centroid positions.
            
        Returns:
        --------
        inertia : float
            Sum of squared distances of samples to their closest cluster center.
        """
        inertia = 0.0
        
        for i in range(X.shape[0]):
            # Distance from point to its assigned centroid
            centroid = centroids[labels[i]]
            inertia += np.sum((X[i] - centroid)**2)
        
        return inertia
    
    def _single_run(self, X):
        """
        Run K-Means algorithm once with a given initialization.
        
        Parameters:
        -----------
        X : numpy.ndarray
            Training data.
            
        Returns:
        --------
        centroids : numpy.ndarray
            Final centroid positions.
        labels : numpy.ndarray
            Final cluster assignments.
        inertia : float
            Final inertia value.
        n_iter : int
            Number of iterations run.
        """
        # Initialize centroids
        centroids = self._init_centroids(X)
        
        # Initialize variables
        previous_inertia = float('inf')
        labels = None
        n_iter = 0
        
        # Main loop
        for iteration in range(self.max_iter):
            # Assign clusters
            labels, min_distances = self._assign_clusters(X, centroids)
            
            # Update centroids
            new_centroids, empty_clusters = self._update_centroids(X, labels)
            
            # Handle empty clusters if any
            if empty_clusters:
                new_centroids, labels = self._handle_empty_clusters(X, labels, empty_clusters)
            
            # Calculate inertia
            inertia = self._calculate_inertia(X, labels, centroids)
            
            # Check for convergence
            inertia_change = previous_inertia - inertia
            if abs(inertia_change) < self.tol * previous_inertia:
                break
                
            # Update for next iteration
            previous_inertia = inertia
            centroids = new_centroids
            n_iter = iteration + 1
        
        return centroids, labels, inertia, n_iter
    
    def fit(self, X):
        """
        Compute K-Means clustering.
        
        Parameters:
        -----------
        X : numpy.ndarray
            Training data.
            
        Returns:
        --------
        self : object
            Fitted estimator.
        """
        # Run K-Means multiple times and select the best result
        best_inertia = float('inf')
        best_centroids = None
        best_labels = None
        best_n_iter = None
        
        for _ in range(self.n_init):
            centroids, labels, inertia, n_iter = self._single_run(X)
            
            if inertia < best_inertia:
                best_centroids = centroids
                best_labels = labels
                best_inertia = inertia
                best_n_iter = n_iter
        
        # Store the best results
        self.centroids = best_centroids
        self.labels = best_labels
        self.inertia = best_inertia
        self.n_iter = best_n_iter
        
        return self
    
    def predict(self, X):
        """
        Predict the closest cluster for each sample in X.
        
        Parameters:
        -----------
        X : numpy.ndarray
            New data to predict.
            
        Returns:
        --------
        labels : numpy.ndarray
            Index of the cluster each sample belongs to.
        """
        if self.centroids is None:
            raise ValueError("Model has not been fitted yet.")
        
        labels, _ = self._assign_clusters(X, self.centroids)
        return labels

## 4. Initialize Centroids with Advanced Techniques

One of the key challenges in K-Means clustering is initializing the centroids. Bad initialization can lead to poor results. We'll implement the K-Means++ algorithm, which selects initial centroids that are far apart from each other, leading to better clustering results.

In [ ]:
# Add the K-Means++ initialization method to our KMeans class
def _kmeans_plus_plus_init(self, X):
    """
    Initialize centroids using the K-Means++ method.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Training data.
        
    Returns:
    --------
    centroids : numpy.ndarray
        Initial positions of centroids.
    """
    n_samples, n_features = X.shape
    centroids = np.zeros((self.n_clusters, n_features))
    
    # Choose the first centroid randomly
    if self.random_state is not None:
        np.random.seed(self.random_state)
        
    first_centroid_idx = np.random.choice(n_samples)
    centroids[0] = X[first_centroid_idx]
    
    # Choose the remaining centroids
    for k in range(1, self.n_clusters):
        # Calculate the squared distances from each point to the nearest existing centroid
        distances = np.zeros(n_samples)
        for i, x in enumerate(X):
            # Find the minimum distance to any existing centroid
            min_dist = float('inf')
            for j in range(k):
                dist = np.sum((x - centroids[j])**2)
                min_dist = min(min_dist, dist)
            distances[i] = min_dist
        
        # Normalize distances to create a probability distribution
        distances_sum = np.sum(distances)
        if distances_sum > 0:
            probabilities = distances / distances_sum
        else:
            # If all points are already centroids, choose randomly
            probabilities = np.ones(n_samples) / n_samples
        
        # Choose the next centroid with probability proportional to squared distance
        next_centroid_idx = np.random.choice(n_samples, p=probabilities)
        centroids[k] = X[next_centroid_idx]
    
    return centroids

# Attach this method to our KMeans class
KMeans._kmeans_plus_plus_init = _kmeans_plus_plus_init

# Create a visualization function to show the initialization difference
def visualize_initializations(X, n_clusters=5, random_state=42):
    """
    Visualize the difference between random and K-Means++ initialization.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to visualize.
    n_clusters : int
        Number of clusters.
    random_state : int
        Random seed for reproducibility.
    """
    # Initialize KMeans with random initialization
    kmeans_random = KMeans(n_clusters=n_clusters, init='random', random_state=random_state, n_init=1)
    random_centroids = kmeans_random._init_centroids(X)
    
    # Initialize KMeans with KMeans++ initialization
    kmeans_plus_plus = KMeans(n_clusters=n_clusters, init='kmeans++', random_state=random_state, n_init=1)
    plus_plus_centroids = kmeans_plus_plus._init_centroids(X)
    
    # Plot the results
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Random initialization
    axes[0].scatter(X[:, 0], X[:, 1], c='gray', alpha=0.5)
    axes[0].scatter(random_centroids[:, 0], random_centroids[:, 1], c='red', marker='X', s=200, label='Centroids')
    axes[0].set_title('Random Initialization')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # KMeans++ initialization
    axes[1].scatter(X[:, 0], X[:, 1], c='gray', alpha=0.5)
    axes[1].scatter(plus_plus_centroids[:, 0], plus_plus_centroids[:, 1], c='red', marker='X', s=200, label='Centroids')
    axes[1].set_title('KMeans++ Initialization')
    axes[1].legend()
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return random_centroids, plus_plus_centroids

# Visualize the two initialization methods
random_centroids, plus_plus_centroids = visualize_initializations(X, n_clusters=5, random_state=42)

print("Random initialization centroid distances:")
random_dists = []
for i in range(len(random_centroids)):
    for j in range(i+1, len(random_centroids)):
        dist = np.linalg.norm(random_centroids[i] - random_centroids[j])
        random_dists.append(dist)
        print(f"Distance between centroids {i} and {j}: {dist:.2f}")
print(f"Average distance: {np.mean(random_dists):.2f}")

print("\nK-Means++ initialization centroid distances:")
plus_plus_dists = []
for i in range(len(plus_plus_centroids)):
    for j in range(i+1, len(plus_plus_centroids)):
        dist = np.linalg.norm(plus_plus_centroids[i] - plus_plus_centroids[j])
        plus_plus_dists.append(dist)
        print(f"Distance between centroids {i} and {j}: {dist:.2f}")
print(f"Average distance: {np.mean(plus_plus_dists):.2f}")

## 5. Iterative Clustering with Convergence Checks

Now let's run our K-Means algorithm on the complex dataset and visualize how it progresses through iterations until convergence.

In [ ]:
def visualize_kmeans_iterations(X, n_clusters=5, init='kmeans++', max_iter=10, random_state=42):
    """
    Visualize the progression of K-Means algorithm through iterations.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to cluster.
    n_clusters : int
        Number of clusters.
    init : str
        Initialization method ('random' or 'kmeans++').
    max_iter : int
        Maximum number of iterations to visualize.
    random_state : int
        Random seed for reproducibility.
    """
    # Create KMeans instance
    kmeans = KMeans(n_clusters=n_clusters, init=init, max_iter=1, n_init=1, random_state=random_state)
    
    # Initialize centroids
    centroids = kmeans._init_centroids(X)
    
    # Create a figure to visualize iterations
    n_rows = (max_iter + 2) // 3  # +2 for initial and final states
    fig, axes = plt.subplots(n_rows, 3, figsize=(18, 6*n_rows))
    axes = axes.flatten()
    
    # Plot initial state
    axes[0].scatter(X[:, 0], X[:, 1], c='gray', alpha=0.5)
    axes[0].scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200)
    axes[0].set_title('Initial Centroids')
    axes[0].grid(alpha=0.3)
    
    # Variables to track convergence
    previous_inertia = float('inf')
    labels = None
    
    # Run iterations
    for iteration in range(max_iter):
        # Assign clusters
        labels, min_distances = kmeans._assign_clusters(X, centroids)
        
        # Update centroids
        new_centroids, empty_clusters = kmeans._update_centroids(X, labels)
        
        # Handle empty clusters if any
        if empty_clusters:
            new_centroids, labels = kmeans._handle_empty_clusters(X, labels, empty_clusters)
        
        # Calculate inertia
        inertia = kmeans._calculate_inertia(X, labels, centroids)
        
        # Plot current state
        axes[iteration + 1].scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', alpha=0.7)
        axes[iteration + 1].scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200)
        for i, center in enumerate(centroids):
            axes[iteration + 1].annotate(f"{i}", (center[0], center[1]), 
                                 fontsize=12, ha='center', va='center',
                                 bbox=dict(boxstyle='circle', fc='white', alpha=0.8))
        
        # Plot arrows showing centroid movement
        for i in range(n_clusters):
            axes[iteration + 1].arrow(centroids[i, 0], centroids[i, 1],
                              new_centroids[i, 0] - centroids[i, 0],
                              new_centroids[i, 1] - centroids[i, 1],
                              head_width=0.5, head_length=0.7, fc='black', ec='black')
        
        # Check for convergence and show on plot
        inertia_change = previous_inertia - inertia
        rel_change = abs(inertia_change) / previous_inertia if previous_inertia > 0 else 0
        converged = rel_change < kmeans.tol
        
        axes[iteration + 1].set_title(f'Iteration {iteration + 1}\nInertia: {inertia:.2f}, Change: {rel_change:.6f}')
        if converged:
            axes[iteration + 1].set_facecolor('#e6ffe6')  # Light green background for convergence
        axes[iteration + 1].grid(alpha=0.3)
        
        # Update for next iteration
        previous_inertia = inertia
        centroids = new_centroids
        
        # Check for early convergence
        if converged:
            break
    
    # Hide any unused subplots
    for i in range(iteration + 2, len(axes)):
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    return centroids, labels, inertia

# Run K-Means with visualization of iterations
final_centroids, final_labels, final_inertia = visualize_kmeans_iterations(
    X, n_clusters=5, init='kmeans++', max_iter=10, random_state=42
)

print(f"Final inertia: {final_inertia:.2f}")
print(f"Number of points in each cluster: {np.bincount(final_labels)}")

# Create a function to analyze convergence speed for different initializations
def analyze_convergence(X, n_clusters=5, max_iter=20, n_runs=5, random_state=42):
    """
    Compare convergence speed between random and K-Means++ initialization.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to cluster.
    n_clusters : int
        Number of clusters.
    max_iter : int
        Maximum number of iterations to run.
    n_runs : int
        Number of runs to average over.
    random_state : int
        Base random seed for reproducibility.
    """
    # Track inertia progress for each method
    random_inertias = np.zeros((n_runs, max_iter))
    kmeans_pp_inertias = np.zeros((n_runs, max_iter))
    
    # Run multiple times to average results
    for run in range(n_runs):
        # Set different seed for each run
        run_seed = random_state + run
        
        # Random initialization
        kmeans_random = KMeans(n_clusters=n_clusters, init='random', max_iter=max_iter, 
                               n_init=1, random_state=run_seed)
        centroids = kmeans_random._init_centroids(X)
        
        previous_inertia = float('inf')
        labels = None
        
        for iteration in range(max_iter):
            # Assign clusters
            labels, _ = kmeans_random._assign_clusters(X, centroids)
            
            # Update centroids
            new_centroids, empty_clusters = kmeans_random._update_centroids(X, labels)
            
            # Handle empty clusters if any
            if empty_clusters:
                new_centroids, labels = kmeans_random._handle_empty_clusters(X, labels, empty_clusters)
            
            # Calculate inertia
            inertia = kmeans_random._calculate_inertia(X, labels, centroids)
            random_inertias[run, iteration] = inertia
            
            # Update for next iteration
            centroids = new_centroids
        
        # K-Means++ initialization
        kmeans_pp = KMeans(n_clusters=n_clusters, init='kmeans++', max_iter=max_iter, 
                           n_init=1, random_state=run_seed)
        centroids = kmeans_pp._init_centroids(X)
        
        previous_inertia = float('inf')
        labels = None
        
        for iteration in range(max_iter):
            # Assign clusters
            labels, _ = kmeans_pp._assign_clusters(X, centroids)
            
            # Update centroids
            new_centroids, empty_clusters = kmeans_pp._update_centroids(X, labels)
            
            # Handle empty clusters if any
            if empty_clusters:
                new_centroids, labels = kmeans_pp._handle_empty_clusters(X, labels, empty_clusters)
            
            # Calculate inertia
            inertia = kmeans_pp._calculate_inertia(X, labels, centroids)
            kmeans_pp_inertias[run, iteration] = inertia
            
            # Update for next iteration
            centroids = new_centroids
    
    # Calculate average inertia across runs
    avg_random_inertias = np.mean(random_inertias, axis=0)
    avg_kmeans_pp_inertias = np.mean(kmeans_pp_inertias, axis=0)
    
    # Plot convergence comparison
    plt.figure(figsize=(12, 6))
    plt.plot(range(1, max_iter + 1), avg_random_inertias, 'o-', label='Random Init')
    plt.plot(range(1, max_iter + 1), avg_kmeans_pp_inertias, 's-', label='K-Means++ Init')
    plt.xlabel('Iteration')
    plt.ylabel('Inertia')
    plt.title('Convergence Speed: Random vs. K-Means++ Initialization')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.yscale('log')  # Log scale to better see differences
    plt.show()
    
    # Calculate number of iterations to converge (when inertia change < 1%)
    random_converge_iter = max_iter
    kmeans_pp_converge_iter = max_iter
    
    for i in range(1, max_iter):
        # Check random init convergence
        random_change = (avg_random_inertias[i-1] - avg_random_inertias[i]) / avg_random_inertias[i-1]
        if random_change < 0.01 and random_converge_iter == max_iter:
            random_converge_iter = i
        
        # Check k-means++ init convergence
        kmeans_pp_change = (avg_kmeans_pp_inertias[i-1] - avg_kmeans_pp_inertias[i]) / avg_kmeans_pp_inertias[i-1]
        if kmeans_pp_change < 0.01 and kmeans_pp_converge_iter == max_iter:
            kmeans_pp_converge_iter = i
    
    print(f"Average iterations to converge (1% threshold):")
    print(f"Random Initialization: {random_converge_iter}")
    print(f"K-Means++ Initialization: {kmeans_pp_converge_iter}")
    print(f"Final inertia (average):")
    print(f"Random Initialization: {avg_random_inertias[-1]:.2f}")
    print(f"K-Means++ Initialization: {avg_kmeans_pp_inertias[-1]:.2f}")

# Analyze convergence for different initialization methods
analyze_convergence(X, n_clusters=5, max_iter=20, n_runs=5, random_state=42)

## 6. Handle Empty Clusters and Edge Cases

K-Means can encounter several edge cases in complex datasets:
1. Empty clusters - when no points are assigned to a centroid
2. Outliers distorting clusters
3. Duplicate centroids
4. Varying cluster densities and sizes

Let's demonstrate how our implementation handles these cases.

In [ ]:
# Create a dataset that's likely to produce empty clusters
def create_challenging_dataset(n_samples=1000, n_clusters=5, random_state=42):
    """
    Create a dataset that's likely to produce empty clusters.
    
    Parameters:
    -----------
    n_samples : int
        Number of samples in the dataset.
    n_clusters : int
        Number of target clusters.
    random_state : int
        Random seed for reproducibility.
    
    Returns:
    --------
    X : numpy.ndarray
        The generated dataset.
    """
    np.random.seed(random_state)
    
    # Create a few dense clusters
    n_dense_clusters = n_clusters - 2
    X = []
    
    # Create dense clusters close together
    centers = []
    for i in range(n_dense_clusters):
        center = np.random.uniform(-5, 5, size=2)
        centers.append(center)
        # Create points around this center
        cluster_size = int(n_samples * 0.8 / n_dense_clusters)
        cluster_points = np.random.normal(loc=center, scale=0.5, size=(cluster_size, 2))
        X.append(cluster_points)
    
    # Create a few distant points
    n_distant = int(n_samples * 0.2)
    distant_points = np.random.uniform(-20, 20, size=(n_distant, 2))
    X.append(distant_points)
    
    # Combine all points and shuffle
    X = np.vstack(X)
    np.random.shuffle(X)
    
    return X

# Create the challenging dataset
challenging_X = create_challenging_dataset(n_samples=1000, n_clusters=5, random_state=42)

# Visualize this dataset
plt.figure(figsize=(10, 8))
plt.scatter(challenging_X[:, 0], challenging_X[:, 1], c='gray', alpha=0.7)
plt.title('Challenging Dataset for K-Means')
plt.grid(alpha=0.3)
plt.show()

# Define a function to demonstrate empty cluster handling
def demonstrate_empty_cluster_handling(X, n_clusters=5, random_state=42):
    """
    Demonstrate how our K-Means implementation handles empty clusters.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to cluster.
    n_clusters : int
        Number of clusters.
    random_state : int
        Random seed for reproducibility.
    
    Returns:
    --------
    kmeans : KMeans
        The fitted KMeans object.
    """
    # Create a standard K-Means instance with random initialization
    # (more likely to produce empty clusters)
    kmeans = KMeans(n_clusters=n_clusters, init='random', max_iter=20, 
                    n_init=1, random_state=random_state)
    
    # Initialize the centroids in a way that's likely to create an empty cluster
    # Place one centroid far from all data points
    centroids = kmeans._init_centroids(X)
    centroids[0] = np.array([30, 30])  # Place one centroid far away
    
    # First iteration: Create visualization of initial state
    plt.figure(figsize=(18, 6))
    
    # Plot 1: Initial centroids
    plt.subplot(1, 3, 1)
    plt.scatter(X[:, 0], X[:, 1], c='gray', alpha=0.5)
    plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200)
    plt.title('Initial Centroids (one far away)')
    plt.grid(alpha=0.3)
    
    # Assign clusters
    labels, _ = kmeans._assign_clusters(X, centroids)
    
    # Plot 2: Cluster assignments
    plt.subplot(1, 3, 2)
    plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', alpha=0.7)
    plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200)
    
    # Check for empty clusters
    empty_clusters = []
    for k in range(n_clusters):
        if np.sum(labels == k) == 0:
            empty_clusters.append(k)
    
    if empty_clusters:
        plt.title(f'Cluster Assignments\nEmpty clusters: {empty_clusters}')
    else:
        plt.title('Cluster Assignments\nNo empty clusters')
    plt.grid(alpha=0.3)
    
    # Update centroids
    new_centroids, empty_clusters = kmeans._update_centroids(X, labels)
    
    # Handle empty clusters
    if empty_clusters:
        new_centroids, new_labels = kmeans._handle_empty_clusters(X, labels, empty_clusters)
    else:
        new_labels = labels
    
    # Plot 3: After handling empty clusters
    plt.subplot(1, 3, 3)
    plt.scatter(X[:, 0], X[:, 1], c=new_labels, cmap='viridis', alpha=0.7)
    plt.scatter(new_centroids[:, 0], new_centroids[:, 1], c='red', marker='X', s=200)
    plt.title('After Handling Empty Clusters')
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Run the full algorithm from this initial state
    centroids = new_centroids
    labels = new_labels
    
    # Main loop for remaining iterations
    for iteration in range(1, kmeans.max_iter):
        # Assign clusters
        labels, _ = kmeans._assign_clusters(X, centroids)
        
        # Update centroids
        new_centroids, empty_clusters = kmeans._update_centroids(X, labels)
        
        # Handle empty clusters
        if empty_clusters:
            new_centroids, labels = kmeans._handle_empty_clusters(X, labels, empty_clusters)
        
        # Check for convergence (simplified)
        if np.allclose(centroids, new_centroids, atol=1e-4):
            break
            
        centroids = new_centroids
    
    # Final state visualization
    plt.figure(figsize=(10, 8))
    plt.scatter(X[:, 0], X[:, 1], c=labels, cmap='viridis', alpha=0.7)
    plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200)
    plt.title(f'Final Clustering after {iteration+1} Iterations')
    plt.grid(alpha=0.3)
    plt.show()
    
    print(f"Final number of points in each cluster: {np.bincount(labels)}")
    
    return kmeans

# Demonstrate how our algorithm handles empty clusters
_ = demonstrate_empty_cluster_handling(challenging_X, n_clusters=5, random_state=42)

# Function to handle outliers in K-Means
def demonstrate_outlier_handling(X, n_clusters=5, random_state=42):
    """
    Demonstrate the effect of outliers on K-Means and how to mitigate it.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to cluster.
    n_clusters : int
        Number of clusters.
    random_state : int
        Random seed for reproducibility.
    """
    # Add some extreme outliers to the dataset
    X_with_outliers = X.copy()
    n_outliers = 10
    outliers = np.random.uniform(-50, 50, size=(n_outliers, 2))
    X_with_outliers = np.vstack([X_with_outliers, outliers])
    
    # Create standard K-Means
    kmeans_standard = KMeans(n_clusters=n_clusters, init='kmeans++', 
                             n_init=1, random_state=random_state)
    kmeans_standard.fit(X_with_outliers)
    
    # Create a visualization
    plt.figure(figsize=(18, 6))
    
    # Plot 1: Original Data with Outliers
    plt.subplot(1, 3, 1)
    plt.scatter(X_with_outliers[:, 0], X_with_outliers[:, 1], c='gray', alpha=0.7)
    plt.title('Dataset with Outliers')
    plt.grid(alpha=0.3)
    
    # Plot 2: Standard K-Means Result
    plt.subplot(1, 3, 2)
    plt.scatter(X_with_outliers[:, 0], X_with_outliers[:, 1], 
                c=kmeans_standard.labels_, cmap='viridis', alpha=0.7)
    plt.scatter(kmeans_standard.centroids[:, 0], kmeans_standard.centroids[:, 1], 
                c='red', marker='X', s=200)
    plt.title('Standard K-Means\n(affected by outliers)')
    plt.grid(alpha=0.3)
    
    # Approach to handle outliers: Pre-filtering
    # Calculate distance to nearest neighbor
    from sklearn.neighbors import NearestNeighbors
    nn = NearestNeighbors(n_neighbors=2)
    nn.fit(X_with_outliers)
    distances, _ = nn.kneighbors(X_with_outliers)
    
    # Use the distance to second nearest neighbor
    outlier_scores = distances[:, 1]
    
    # Define outliers as points with nearest neighbor distance > 3 std dev
    threshold = np.mean(outlier_scores) + 3 * np.std(outlier_scores)
    outlier_mask = outlier_scores > threshold
    
    # Filter out outliers
    X_filtered = X_with_outliers[~outlier_mask]
    
    # Run K-Means on filtered dataset
    kmeans_filtered = KMeans(n_clusters=n_clusters, init='kmeans++', 
                             n_init=1, random_state=random_state)
    kmeans_filtered.fit(X_filtered)
    
    # Predict cluster for all points including outliers
    outlier_labels = kmeans_filtered.predict(X_with_outliers)
    
    # Plot 3: K-Means with Outlier Handling
    plt.subplot(1, 3, 3)
    # Regular points
    plt.scatter(X_with_outliers[~outlier_mask, 0], X_with_outliers[~outlier_mask, 1], 
                c=outlier_labels[~outlier_mask], cmap='viridis', alpha=0.7)
    # Outliers
    plt.scatter(X_with_outliers[outlier_mask, 0], X_with_outliers[outlier_mask, 1], 
                c='red', marker='x', s=100, alpha=0.7)
    # Centroids
    plt.scatter(kmeans_filtered.centroids[:, 0], kmeans_filtered.centroids[:, 1], 
                c='black', marker='X', s=200)
    
    plt.title('K-Means with Outlier Handling\n(Outliers marked as X)')
    plt.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Number of detected outliers: {np.sum(outlier_mask)}")
    print(f"True number of added outliers: {n_outliers}")

# Demonstrate outlier handling
demonstrate_outlier_handling(challenging_X, n_clusters=5, random_state=42)

## 7. Visualize Clustering Results

Let's create more advanced visualizations of our clustering results, including:
1. Decision boundaries between clusters
2. 3D visualization for higher dimensions
3. Cluster size and density information

In [ ]:
# Apply KMeans to our original dataset
kmeans = KMeans(n_clusters=5, init='kmeans++', n_init=10, random_state=42)
kmeans.fit(X)

# Create a function to visualize decision boundaries
def plot_decision_boundaries(X, kmeans, boundary_step=0.05):
    """
    Visualize decision boundaries for K-Means clustering.
    
    Parameters:
    -----------
    X : numpy.ndarray
        The dataset that was clustered.
    kmeans : KMeans
        Fitted KMeans instance.
    boundary_step : float
        Step size for the meshgrid.
    """
    # Create a meshgrid
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, boundary_step),
                         np.arange(y_min, y_max, boundary_step))
    
    # Predict cluster for each point in the meshgrid
    Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    
    # Create figure
    plt.figure(figsize=(12, 10))
    
    # Plot the decision boundary
    plt.contourf(xx, yy, Z, alpha=0.3, cmap='viridis')
    
    # Plot the original data points
    plt.scatter(X[:, 0], X[:, 1], c=kmeans.labels_, cmap='viridis', edgecolor='k', alpha=0.7)
    
    # Plot the centroids
    plt.scatter(kmeans.centroids[:, 0], kmeans.centroids[:, 1], 
                c='red', marker='X', s=200, edgecolor='k')
    
    for i, centroid in enumerate(kmeans.centroids):
        plt.annotate(f"Cluster {i}", (centroid[0], centroid[1]), 
                     fontsize=12, ha='center', va='center',
                     bbox=dict(boxstyle='round,pad=0.5', fc='yellow', alpha=0.7))
    
    plt.title('K-Means Clustering with Decision Boundaries')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()

# Visualize decision boundaries
plot_decision_boundaries(X, kmeans, boundary_step=0.1)

# Create a 3D visualization (we need to generate 3D data first)
def generate_3d_data(n_samples=1000, n_clusters=5, random_state=42):
    """
    Generate 3D data for visualization.
    
    Parameters:
    -----------
    n_samples : int
        Number of samples.
    n_clusters : int
        Number of clusters.
    random_state : int
        Random seed.
        
    Returns:
    --------
    X : numpy.ndarray
        3D dataset.
    true_labels : numpy.ndarray
        True cluster labels.
    """
    np.random.seed(random_state)
    
    # Generate cluster centers
    centers = np.random.uniform(-10, 10, size=(n_clusters, 3))
    
    # Initialize arrays
    X = np.zeros((n_samples, 3))
    true_labels = np.zeros(n_samples, dtype=int)
    
    # Generate points around centers
    samples_per_cluster = n_samples // n_clusters
    for i in range(n_clusters):
        start_idx = i * samples_per_cluster
        end_idx = (i + 1) * samples_per_cluster if i < n_clusters - 1 else n_samples
        
        # Random spread for this cluster
        spread = np.random.uniform(0.5, 2.0)
        
        # Generate points
        X[start_idx:end_idx] = np.random.normal(centers[i], spread, size=(end_idx - start_idx, 3))
        true_labels[start_idx:end_idx] = i
    
    # Shuffle the data
    idx = np.random.permutation(n_samples)
    X = X[idx]
    true_labels = true_labels[idx]
    
    return X, true_labels

# Generate 3D data
X_3d, true_labels_3d = generate_3d_data(n_samples=1000, n_clusters=5, random_state=42)

# Apply KMeans to 3D data
kmeans_3d = KMeans(n_clusters=5, init='kmeans++', n_init=10, random_state=42)
kmeans_3d.fit(X_3d)

# Create 3D visualization
def plot_3d_clusters(X, labels, centroids, title="3D K-Means Clustering"):
    """
    Create a 3D visualization of clustering results.
    
    Parameters:
    -----------
    X : numpy.ndarray
        3D dataset.
    labels : numpy.ndarray
        Cluster assignments.
    centroids : numpy.ndarray
        Centroids of clusters.
    title : str
        Plot title.
    """
    fig = plt.figure(figsize=(12, 10))
    ax = fig.add_subplot(111, projection='3d')
    
    # Plot data points
    scatter = ax.scatter(X[:, 0], X[:, 1], X[:, 2], 
                         c=labels, cmap='viridis', alpha=0.7, s=30)
    
    # Plot centroids
    ax.scatter(centroids[:, 0], centroids[:, 1], centroids[:, 2],
               c='red', marker='X', s=200, edgecolor='k')
    
    # Add cluster annotations
    for i, centroid in enumerate(centroids):
        ax.text(centroid[0], centroid[1], centroid[2], f"Cluster {i}",
                fontsize=12, ha='center', va='center')
    
    ax.set_title(title)
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')
    ax.set_zlabel('Feature 3')
    
    # Add a color bar
    cbar = fig.colorbar(scatter, ax=ax, pad=0.1)
    cbar.set_label('Cluster')
    
    plt.tight_layout()
    plt.show()

# Visualize 3D clusters
plot_3d_clusters(X_3d, kmeans_3d.labels_, kmeans_3d.centroids)

# Create a function to visualize cluster characteristics
def visualize_cluster_characteristics(X, kmeans):
    """
    Visualize characteristics of clusters including size, density, and spread.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset that was clustered.
    kmeans : KMeans
        Fitted KMeans instance.
    """
    # Calculate cluster sizes
    cluster_sizes = np.bincount(kmeans.labels_)
    
    # Calculate cluster densities (average distance to centroid)
    densities = []
    spreads = []
    
    for i in range(kmeans.n_clusters):
        # Get points in this cluster
        cluster_points = X[kmeans.labels_ == i]
        
        if len(cluster_points) > 0:
            # Calculate distances to centroid
            centroid = kmeans.centroids[i]
            distances = np.sqrt(np.sum((cluster_points - centroid)**2, axis=1))
            
            # Average distance = density (lower means more dense)
            densities.append(np.mean(distances))
            
            # Standard deviation of distances = spread
            spreads.append(np.std(distances))
        else:
            densities.append(0)
            spreads.append(0)
    
    # Create figure
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Plot cluster sizes
    axes[0].bar(range(kmeans.n_clusters), cluster_sizes, color='skyblue')
    axes[0].set_title('Cluster Sizes')
    axes[0].set_xlabel('Cluster')
    axes[0].set_ylabel('Number of Points')
    axes[0].set_xticks(range(kmeans.n_clusters))
    axes[0].grid(axis='y', alpha=0.3)
    
    # Plot cluster densities
    axes[1].bar(range(kmeans.n_clusters), densities, color='lightgreen')
    axes[1].set_title('Cluster Densities\n(Average Distance to Centroid)')
    axes[1].set_xlabel('Cluster')
    axes[1].set_ylabel('Average Distance')
    axes[1].set_xticks(range(kmeans.n_clusters))
    axes[1].grid(axis='y', alpha=0.3)
    
    # Plot cluster spreads
    axes[2].bar(range(kmeans.n_clusters), spreads, color='salmon')
    axes[2].set_title('Cluster Spreads\n(Standard Deviation of Distances)')
    axes[2].set_xlabel('Cluster')
    axes[2].set_ylabel('Standard Deviation')
    axes[2].set_xticks(range(kmeans.n_clusters))
    axes[2].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("Cluster Statistics:")
    for i in range(kmeans.n_clusters):
        print(f"Cluster {i}:")
        print(f"  Size: {cluster_sizes[i]} points")
        print(f"  Density (avg distance to centroid): {densities[i]:.4f}")
        print(f"  Spread (std dev of distances): {spreads[i]:.4f}")
        print()

# Visualize cluster characteristics
visualize_cluster_characteristics(X, kmeans)

## 8. Evaluate Clustering Performance with Metrics

To evaluate the performance of K-Means clustering, we can use several metrics:

1. **Inertia**: Sum of squared distances to centroids (lower is better, but decreases with more clusters)
2. **Silhouette Score**: Measures how similar points are to their own cluster vs other clusters (-1 to 1, higher is better)
3. **Calinski-Harabasz Index**: Ratio of between-cluster to within-cluster dispersion (higher is better)
4. **Davies-Bouldin Index**: Average similarity of each cluster with its most similar cluster (lower is better)
5. **Adjusted Rand Index**: Measures similarity between two clusterings (useful when true labels are known)

In [ ]:
# Import additional evaluation metrics
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Define a function to compute and visualize the Elbow Method
def plot_elbow_method(X, max_k=10, random_state=42):
    """
    Plot the Elbow Method to find the optimal number of clusters.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to cluster.
    max_k : int
        Maximum number of clusters to try.
    random_state : int
        Random seed for reproducibility.
    """
    inertias = []
    
    for k in range(1, max_k + 1):
        if k == 1:
            # For k=1, inertia is just the sum of squared distances to the mean
            centroid = np.mean(X, axis=0)
            inertia = np.sum(np.sum((X - centroid)**2, axis=1))
            inertias.append(inertia)
        else:
            # For k>1, use our KMeans implementation
            kmeans = KMeans(n_clusters=k, init='kmeans++', n_init=10, random_state=random_state)
            kmeans.fit(X)
            inertias.append(kmeans.inertia)
    
    # Plot the elbow curve
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, max_k + 1), inertias, 'o-', color='blue')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Inertia')
    plt.title('Elbow Method for Optimal k')
    plt.grid(alpha=0.3)
    
    # Add annotations for the "elbow" point
    # Calculate the angle between consecutive points
    angles = []
    for i in range(1, len(inertias) - 1):
        x1, y1 = i, inertias[i]
        x0, y0 = i - 1, inertias[i - 1]
        x2, y2 = i + 1, inertias[i + 1]
        
        # Calculate vectors
        v1 = np.array([x1 - x0, y1 - y0])
        v2 = np.array([x2 - x1, y2 - y1])
        
        # Normalize vectors
        v1 = v1 / np.linalg.norm(v1)
        v2 = v2 / np.linalg.norm(v2)
        
        # Calculate angle using dot product
        angle = np.arccos(np.clip(np.dot(v1, v2), -1.0, 1.0))
        angles.append(angle)
    
    # Potential elbow point is where angle is largest
    elbow_idx = np.argmax(angles) + 1  # +1 because we start from the second point
    elbow_k = elbow_idx + 1  # +1 because k starts from 1
    
    plt.annotate(f'Potential elbow at k={elbow_k}',
                 xy=(elbow_k, inertias[elbow_idx]),
                 xytext=(elbow_k + 1, inertias[elbow_idx] * 1.2),
                 arrowprops=dict(facecolor='black', shrink=0.05, width=2))
    
    plt.tight_layout()
    plt.show()
    
    return inertias

# Plot elbow method
inertias = plot_elbow_method(X, max_k=10, random_state=42)

# Define a function to compute and visualize the Silhouette Method
def plot_silhouette_analysis(X, max_k=10, random_state=42):
    """
    Perform silhouette analysis to find the optimal number of clusters.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to cluster.
    max_k : int
        Maximum number of clusters to try.
    random_state : int
        Random seed for reproducibility.
    """
    silhouette_scores = []
    k_values = range(2, max_k + 1)  # Silhouette requires at least 2 clusters
    
    for k in k_values:
        # Cluster the data
        kmeans = KMeans(n_clusters=k, init='kmeans++', n_init=10, random_state=random_state)
        kmeans.fit(X)
        
        # Calculate silhouette score
        score = silhouette_score(X, kmeans.labels_)
        silhouette_scores.append(score)
    
    # Plot silhouette scores
    plt.figure(figsize=(10, 6))
    plt.plot(k_values, silhouette_scores, 'o-', color='green')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Silhouette Score')
    plt.title('Silhouette Analysis for Optimal k')
    plt.grid(alpha=0.3)
    
    # Mark the maximum silhouette score
    best_k_idx = np.argmax(silhouette_scores)
    best_k = k_values[best_k_idx]
    best_score = silhouette_scores[best_k_idx]
    
    plt.annotate(f'Best k={best_k}, Score={best_score:.4f}',
                 xy=(best_k, best_score),
                 xytext=(best_k + 1, best_score - 0.05),
                 arrowprops=dict(facecolor='black', shrink=0.05, width=2))
    
    plt.tight_layout()
    plt.show()
    
    return silhouette_scores, best_k

# Plot silhouette analysis
silhouette_scores, best_k = plot_silhouette_analysis(X, max_k=10, random_state=42)

# Evaluate the optimal clustering using multiple metrics
def evaluate_clustering(X, true_labels=None, best_k=None, random_state=42):
    """
    Evaluate clustering performance using multiple metrics.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to cluster.
    true_labels : numpy.ndarray or None
        True cluster labels if available.
    best_k : int or None
        Number of clusters to use. If None, use 5.
    random_state : int
        Random seed for reproducibility.
    """
    if best_k is None:
        best_k = 5
    
    # Fit KMeans with the optimal k
    kmeans = KMeans(n_clusters=best_k, init='kmeans++', n_init=10, random_state=random_state)
    kmeans.fit(X)
    
    # Calculate evaluation metrics
    metrics = {
        'Inertia': kmeans.inertia,
        'Silhouette Score': silhouette_score(X, kmeans.labels_),
        'Calinski-Harabasz Index': calinski_harabasz_score(X, kmeans.labels_),
        'Davies-Bouldin Index': davies_bouldin_score(X, kmeans.labels_)
    }
    
    if true_labels is not None:
        metrics['Adjusted Rand Index'] = adjusted_rand_score(true_labels, kmeans.labels_)
    
    # Print metrics
    print(f"Evaluation Metrics for k={best_k}:")
    for name, value in metrics.items():
        print(f"{name}: {value:.4f}")
    
    return kmeans, metrics

# Evaluate clustering with the optimal k
optimal_kmeans, metrics = evaluate_clustering(X, true_labels=true_labels, best_k=best_k, random_state=42)

# Compare evaluation metrics for different k values
def compare_metrics_for_different_k(X, true_labels=None, max_k=10, random_state=42):
    """
    Compare different evaluation metrics for various k values.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to cluster.
    true_labels : numpy.ndarray or None
        True cluster labels if available.
    max_k : int
        Maximum number of clusters to try.
    random_state : int
        Random seed for reproducibility.
    """
    k_values = range(2, max_k + 1)
    
    # Initialize metric arrays
    inertias = []
    silhouette_scores = []
    ch_scores = []
    db_scores = []
    ari_scores = []
    
    for k in k_values:
        # Fit KMeans
        kmeans = KMeans(n_clusters=k, init='kmeans++', n_init=5, random_state=random_state)
        kmeans.fit(X)
        
        # Calculate metrics
        inertias.append(kmeans.inertia)
        silhouette_scores.append(silhouette_score(X, kmeans.labels_))
        ch_scores.append(calinski_harabasz_score(X, kmeans.labels_))
        db_scores.append(davies_bouldin_score(X, kmeans.labels_))
        
        if true_labels is not None:
            ari_scores.append(adjusted_rand_score(true_labels, kmeans.labels_))
    
    # Plot the metrics
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Inertia (lower is better, but always decreases with k)
    axes[0, 0].plot(k_values, inertias, 'o-', color='blue')
    axes[0, 0].set_xlabel('Number of Clusters (k)')
    axes[0, 0].set_ylabel('Inertia')
    axes[0, 0].set_title('Inertia vs. Number of Clusters')
    axes[0, 0].grid(alpha=0.3)
    
    # Silhouette Score (higher is better, range: -1 to 1)
    axes[0, 1].plot(k_values, silhouette_scores, 'o-', color='green')
    axes[0, 1].set_xlabel('Number of Clusters (k)')
    axes[0, 1].set_ylabel('Silhouette Score')
    axes[0, 1].set_title('Silhouette Score vs. Number of Clusters')
    axes[0, 1].grid(alpha=0.3)
    
    # Calinski-Harabasz Index (higher is better)
    axes[1, 0].plot(k_values, ch_scores, 'o-', color='purple')
    axes[1, 0].set_xlabel('Number of Clusters (k)')
    axes[1, 0].set_ylabel('Calinski-Harabasz Index')
    axes[1, 0].set_title('Calinski-Harabasz Index vs. Number of Clusters')
    axes[1, 0].grid(alpha=0.3)
    
    # Davies-Bouldin Index (lower is better)
    axes[1, 1].plot(k_values, db_scores, 'o-', color='red')
    axes[1, 1].set_xlabel('Number of Clusters (k)')
    axes[1, 1].set_ylabel('Davies-Bouldin Index')
    axes[1, 1].set_title('Davies-Bouldin Index vs. Number of Clusters')
    axes[1, 1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # If true labels are available, plot ARI
    if true_labels is not None:
        plt.figure(figsize=(10, 6))
        plt.plot(k_values, ari_scores, 'o-', color='orange')
        plt.xlabel('Number of Clusters (k)')
        plt.ylabel('Adjusted Rand Index')
        plt.title('Adjusted Rand Index vs. Number of Clusters')
        plt.grid(alpha=0.3)
        plt.tight_layout()
        plt.show()
        
        # Find k with highest ARI
        best_k_ari = k_values[np.argmax(ari_scores)]
        print(f"Best k according to Adjusted Rand Index: {best_k_ari}")
    
    # Find best k according to each metric
    best_k_inertia = None  # No clear criterion for inertia
    best_k_silhouette = k_values[np.argmax(silhouette_scores)]
    best_k_ch = k_values[np.argmax(ch_scores)]
    best_k_db = k_values[np.argmin(db_scores)]
    
    print("Best k according to different metrics:")
    print(f"Silhouette Score: {best_k_silhouette}")
    print(f"Calinski-Harabasz Index: {best_k_ch}")
    print(f"Davies-Bouldin Index: {best_k_db}")
    
    return {
        'inertias': inertias,
        'silhouette_scores': silhouette_scores,
        'ch_scores': ch_scores,
        'db_scores': db_scores,
        'ari_scores': ari_scores if true_labels is not None else None
    }

# Compare metrics for different k values
metric_comparison = compare_metrics_for_different_k(X, true_labels=true_labels, max_k=10, random_state=42)

## 9. Compare with scikit-learn KMeans Implementation

Finally, let's compare our custom K-Means implementation with scikit-learn's implementation in terms of:
1. Clustering results
2. Performance (execution time)
3. Centroid positions
4. Evaluation metrics

In [ ]:
import time
from sklearn.cluster import KMeans as SklearnKMeans

# Define a function to compare our implementation with scikit-learn
def compare_kmeans_implementations(X, n_clusters=5, random_state=42):
    """
    Compare our custom KMeans implementation with scikit-learn's implementation.
    
    Parameters:
    -----------
    X : numpy.ndarray
        Dataset to cluster.
    n_clusters : int
        Number of clusters.
    random_state : int
        Random seed for reproducibility.
    """
    # Our implementation
    start_time = time.time()
    custom_kmeans = KMeans(n_clusters=n_clusters, init='kmeans++', n_init=10, random_state=random_state)
    custom_kmeans.fit(X)
    custom_time = time.time() - start_time
    
    # scikit-learn implementation
    start_time = time.time()
    sklearn_kmeans = SklearnKMeans(n_clusters=n_clusters, init='k-means++', n_init=10, random_state=random_state)
    sklearn_kmeans.fit(X)
    sklearn_time = time.time() - start_time
    
    # Compare clustering results
    custom_labels = custom_kmeans.labels_
    sklearn_labels = sklearn_kmeans.labels_
    
    # Calculate agreement between clusterings
    agreement_score = adjusted_rand_score(custom_labels, sklearn_labels)
    
    # Print comparison
    print("=== Performance Comparison ===")
    print(f"Our Implementation: {custom_time:.6f} seconds")
    print(f"scikit-learn: {sklearn_time:.6f} seconds")
    print(f"Speed Ratio (sklearn/custom): {sklearn_time/custom_time:.2f}x")
    print()
    
    print("=== Clustering Agreement ===")
    print(f"Adjusted Rand Index: {agreement_score:.6f}")
    print(f"(1.0 would mean identical clusterings, accounting for label permutation)")
    print()
    
    print("=== Inertia Comparison ===")
    print(f"Our Implementation: {custom_kmeans.inertia:.2f}")
    print(f"scikit-learn: {sklearn_kmeans.inertia_:.2f}")
    print(f"Difference: {abs(custom_kmeans.inertia - sklearn_kmeans.inertia_):.2f}")
    print()
    
    # Compare evaluation metrics
    custom_silhouette = silhouette_score(X, custom_labels)
    sklearn_silhouette = silhouette_score(X, sklearn_labels)
    
    print("=== Silhouette Score Comparison ===")
    print(f"Our Implementation: {custom_silhouette:.6f}")
    print(f"scikit-learn: {sklearn_silhouette:.6f}")
    print()
    
    # Visualize the clustering differences
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    
    # Our implementation
    axes[0].scatter(X[:, 0], X[:, 1], c=custom_labels, cmap='viridis', alpha=0.7)
    axes[0].scatter(custom_kmeans.centroids[:, 0], custom_kmeans.centroids[:, 1], 
                    c='red', marker='X', s=200, edgecolor='k')
    axes[0].set_title(f'Our Implementation\nInertia: {custom_kmeans.inertia:.2f}, Silhouette: {custom_silhouette:.4f}')
    axes[0].grid(alpha=0.3)
    
    # scikit-learn implementation
    axes[1].scatter(X[:, 0], X[:, 1], c=sklearn_labels, cmap='viridis', alpha=0.7)
    axes[1].scatter(sklearn_kmeans.cluster_centers_[:, 0], sklearn_kmeans.cluster_centers_[:, 1], 
                    c='red', marker='X', s=200, edgecolor='k')
    axes[1].set_title(f'scikit-learn Implementation\nInertia: {sklearn_kmeans.inertia_:.2f}, Silhouette: {sklearn_silhouette:.4f}')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return custom_kmeans, sklearn_kmeans

# Compare our implementation with scikit-learn
custom_kmeans, sklearn_kmeans = compare_kmeans_implementations(X, n_clusters=5, random_state=42)

# Perform a detailed centroid comparison
def compare_centroids(custom_kmeans, sklearn_kmeans):
    """
    Compare centroids between our implementation and scikit-learn.
    
    Parameters:
    -----------
    custom_kmeans : KMeans
        Our fitted KMeans instance.
    sklearn_kmeans : sklearn.cluster.KMeans
        Fitted scikit-learn KMeans instance.
    """
    # Get centroids
    custom_centroids = custom_kmeans.centroids
    sklearn_centroids = sklearn_kmeans.cluster_centers_
    
    # Calculate distances between each pair of centroids
    distances = np.zeros((custom_centroids.shape[0], sklearn_centroids.shape[0]))
    for i in range(custom_centroids.shape[0]):
        for j in range(sklearn_centroids.shape[0]):
            distances[i, j] = np.sqrt(np.sum((custom_centroids[i] - sklearn_centroids[j])**2))
    
    # Find best matching centroids
    from scipy.optimize import linear_sum_assignment
    row_ind, col_ind = linear_sum_assignment(distances)
    
    # Visualize the matching
    plt.figure(figsize=(10, 8))
    plt.scatter(X[:, 0], X[:, 1], c='lightgray', alpha=0.3)
    
    # Plot centroids with connections
    for i, j in zip(row_ind, col_ind):
        plt.scatter(custom_centroids[i, 0], custom_centroids[i, 1], c='blue', marker='o', s=150, label='Our' if i == 0 else "")
        plt.scatter(sklearn_centroids[j, 0], sklearn_centroids[j, 1], c='green', marker='s', s=150, label='sklearn' if j == 0 else "")
        plt.plot([custom_centroids[i, 0], sklearn_centroids[j, 0]],
                 [custom_centroids[i, 1], sklearn_centroids[j, 1]], 'k--', alpha=0.5)
        
        # Annotate the centroids
        plt.annotate(f"Our {i}", (custom_centroids[i, 0], custom_centroids[i, 1]), 
                     xytext=(10, 5), textcoords='offset points')
        plt.annotate(f"sklearn {j}", (sklearn_centroids[j, 0], sklearn_centroids[j, 1]), 
                     xytext=(10, -10), textcoords='offset points')
        
        # Print the distance
        midpoint = ((custom_centroids[i, 0] + sklearn_centroids[j, 0])/2,
                    (custom_centroids[i, 1] + sklearn_centroids[j, 1])/2)
        plt.annotate(f"{distances[i, j]:.4f}", midpoint, 
                     xytext=(0, 0), textcoords='offset points', ha='center')
    
    plt.title('Centroid Comparison: Our Implementation vs. scikit-learn')
    plt.legend()
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    # Print detailed comparison
    print("=== Centroid Distance Comparison ===")
    total_distance = 0
    for i, j in zip(row_ind, col_ind):
        dist = distances[i, j]
        total_distance += dist
        print(f"Our Centroid {i} <-> sklearn Centroid {j}: Distance = {dist:.6f}")
    
    print(f"\nAverage Centroid Distance: {total_distance/len(row_ind):.6f}")
    
    return distances, row_ind, col_ind

# Compare centroids between implementations
distances, row_ind, col_ind = compare_centroids(custom_kmeans, sklearn_kmeans)

# Compare on a more challenging dataset
def compare_on_challenging_dataset(n_samples=1000, n_features=10, n_clusters=5, random_state=42):
    """
    Compare implementations on a more challenging high-dimensional dataset.
    
    Parameters:
    -----------
    n_samples : int
        Number of samples.
    n_features : int
        Number of features (dimensions).
    n_clusters : int
        Number of clusters.
    random_state : int
        Random seed for reproducibility.
    """
    # Create a high-dimensional dataset
    np.random.seed(random_state)
    
    # Generate cluster centers
    centers = np.random.uniform(-10, 10, size=(n_clusters, n_features))
    
    # Generate points
    X_high_dim = np.zeros((n_samples, n_features))
    true_labels = np.zeros(n_samples, dtype=int)
    
    samples_per_cluster = n_samples // n_clusters
    for i in range(n_clusters):
        start_idx = i * samples_per_cluster
        end_idx = (i + 1) * samples_per_cluster if i < n_clusters - 1 else n_samples
        
        # Generate points around center with varying spread
        spread = np.random.uniform(0.5, 2.0, size=n_features)
        X_high_dim[start_idx:end_idx] = np.random.normal(
            centers[i], spread, size=(end_idx - start_idx, n_features))
        true_labels[start_idx:end_idx] = i
    
    # Shuffle the data
    idx = np.random.permutation(n_samples)
    X_high_dim = X_high_dim[idx]
    true_labels = true_labels[idx]
    
    print(f"Generated {n_samples} samples with {n_features} features and {n_clusters} clusters")
    
    # Compare performance
    start_time = time.time()
    custom_kmeans = KMeans(n_clusters=n_clusters, init='kmeans++', n_init=5, random_state=random_state)
    custom_kmeans.fit(X_high_dim)
    custom_time = time.time() - start_time
    
    start_time = time.time()
    sklearn_kmeans = SklearnKMeans(n_clusters=n_clusters, init='k-means++', n_init=5, random_state=random_state)
    sklearn_kmeans.fit(X_high_dim)
    sklearn_time = time.time() - start_time
    
    # Print performance comparison
    print("\n=== Performance Comparison (High-Dimensional Data) ===")
    print(f"Our Implementation: {custom_time:.6f} seconds")
    print(f"scikit-learn: {sklearn_time:.6f} seconds")
    print(f"Speed Ratio (sklearn/custom): {sklearn_time/custom_time:.2f}x")
    
    # Compare clustering quality
    custom_ari = adjusted_rand_score(true_labels, custom_kmeans.labels_)
    sklearn_ari = adjusted_rand_score(true_labels, sklearn_kmeans.labels_)
    
    print("\n=== Clustering Quality Comparison ===")
    print(f"Our Implementation ARI: {custom_ari:.6f}")
    print(f"scikit-learn ARI: {sklearn_ari:.6f}")
    
    # Compare inertia
    print("\n=== Inertia Comparison ===")
    print(f"Our Implementation: {custom_kmeans.inertia:.2f}")
    print(f"scikit-learn: {sklearn_kmeans.inertia_:.2f}")
    
    return custom_kmeans, sklearn_kmeans, X_high_dim, true_labels

# Compare on a high-dimensional dataset
high_dim_custom, high_dim_sklearn, X_high_dim, high_dim_true = compare_on_challenging_dataset(
    n_samples=2000, n_features=10, n_clusters=7, random_state=42)

# Final summary and conclusions
print("\n===== Final Comparison Summary =====")
print("1. Implementation Differences:")
print("   - Our implementation includes detailed handling of empty clusters")
print("   - Our implementation has more visualization and analysis capabilities")
print("   - scikit-learn is optimized for performance and has more options")

print("\n2. Performance:")
print("   - scikit-learn is typically faster due to optimized C/C++ code")
print("   - Our implementation is more transparent and educational")

print("\n3. Clustering Quality:")
print("   - Both implementations produce similar clustering results")
print("   - Differences in results are mostly due to different random initializations")
print("   - scikit-learn might converge better due to its optimized implementation")

print("\n4. When to Use Each:")
print("   - Use scikit-learn for production and large datasets")
print("   - Use our implementation for learning and understanding K-Means")
print("   - Our implementation allows easier customization and visualization")

## 10. Conclusion

In this notebook, we've implemented a comprehensive K-Means clustering algorithm from scratch and compared it with scikit-learn's implementation. Here's what we've learned:

1. **K-Means Fundamentals**:
   - The core algorithm is simple but powerful
   - Initialization significantly affects results (K-Means++ vs Random)
   - Convergence typically occurs within 10-20 iterations

2. **Advanced Implementation Aspects**:
   - Handling empty clusters is crucial for robustness
   - Dealing with outliers improves clustering quality
   - Multiple runs with different initializations help find better solutions

3. **Evaluation Techniques**:
   - Elbow method helps determine optimal cluster count
   - Silhouette score provides insight into cluster quality
   - Multiple metrics should be considered for thorough evaluation

4. **Real-world Considerations**:
   - K-Means struggles with non-spherical or unequally sized clusters
   - High-dimensional data presents additional challenges
   - Preprocessing (scaling, outlier removal) is often essential

5. **Custom vs. Library Implementation**:
   - Our implementation offers transparency and educational value
   - scikit-learn offers performance and robustness
   - Both provide similar clustering results

Understanding the intricacies of K-Means implementation equips data scientists with the knowledge to better apply and interpret this widely-used clustering algorithm in real-world scenarios.